## Create instruct dataset with Mixtral

In [ ]:
pip install langchain

### Create a Mixtral 8x7B endpoint

In [2]:
from PIL import Image
import base64
import json
import os

from utils import *

from sagemaker.jumpstart.model import JumpStartModel

smr_client = boto3.client("sagemaker-runtime")

# Set the endpoint name if using the existing endpoint
llm_endpoint_name = None
initial_instance_count = 5
if llm_endpoint_name is None:
    model_id = "huggingface-llm-mixtral-8x7b-instruct"
    model = JumpStartModel(model_id=model_id)
    predictor = model.deploy(initial_instance_count=initial_instance_count)
    llm_endpoint_name = predictor.endpoint

ModuleNotFoundError: No module named 'boto3'

### Prepare dataset

#### Data selection
Only choose the samples used in `llava_v1_5_mix665k`. It contains the following data source:

<table>
    <tr><th>Data</th><th>Size</th></tr>
    <tr><td>LLaVA</td><td>158K</td></tr>
    <tr><td>ShareGPT</td><td>40K</td></tr>
    <tr><td>GQA</td><td>72K</td></tr>
    <tr><td>OKVQA</td><td>9K</td></tr>
    <tr><td>OCRVQA</td><td>80K</td></tr>
    <tr><td>A-OKVQA</td><td>50K</td></tr>
    <tr><td>TextCaps</td><td>22K</td></tr>
    <tr><td>RefCOCO</td><td>30K</td></tr>
    <tr><td>VG</td><td>86K</td></tr>
</table>

In [5]:
dataset_dir = "dataset"

with open(os.path.join(dataset_dir, 'LLaVA-Instruct-150K/llava_v1_5_mix665k.json'), 'r') as f:
    mix665k = json.load(f)

In [6]:
mix665_dict = {}

turn_num = []
for ins_data in mix665k:
    if ins_data['id'] in mix665_dict:
        mix665_dict[ins_data['id']].append(ins_data)
    else:
        mix665_dict[ins_data['id']] = [ins_data]
        turn_num.append(len(ins_data['conversations'])/2)

Choose the dataset

In [7]:
dataset_name = 'coco'

def load_coco(coco_caption_path):
    with open(coco_caption_path, 'r') as f:
        coco_caption = json.load(f)

    coco_cap_new = {}
    for cap in coco_caption['annotations']:
        if cap['image_id'] in coco_cap_new:
            coco_cap_new[cap['image_id']].append(cap['caption'])
        else:
            coco_cap_new[cap['image_id']] = [cap['caption']]
            
    return coco_cap_new

caption_path = os.path.join(dataset_dir, "annotations/captions_train2017.json")
coco_cap_new = load_coco(caption_path)

Filter dataset as`llava_v1_5_mix665k.json` mixting multiple dataset.

In [8]:
filtered_data = {}

for img_id, ann in mix665_dict.items():
    if 'image' in ann[0] and dataset_name in ann[0]['image']:
        filtered_data[str(int(img_id))] = ann
coco_ids = [img_id for img_id in filtered_data.keys()]

Only choose `image_id` existing in `llava_v1_5_mix665k.json` for performance comparison

In [9]:
filtered_coco_caps = {coco_id: coco_cap_new[int(coco_id)] for coco_id in coco_ids if int(coco_id) in coco_cap_new}

In [10]:
len(filtered_coco_caps)

88524

In [11]:
filtered_coco_caps

{'33471': ['A white and red bus driving down a city street.',
  'A transit bus riding past some people in a parking area.',
  'A huge white bus with advertisement on the back is driving down the road.',
  'The city bus is pulled off to the side of the road. ',
  'a big bus is driving down a crowded street'],
 '52846': ['A cat is coming out from some curtains onto a couch.',
  'A cat is standing on the back of a couch.',
  'a couch sitting in a living room with a cat sitting on top',
  'A couch with lots of pillows and a cat sitting on top of it.',
  'A cat was on top of the couch by the red curtains'],
 '334872': ['A couple of people standing in the snow with skis.',
  'A pair of men with backpacks riding skis on a slope.',
  'Two men cross country skiing in the woods ',
  'Two guys on skies on a trail in the woods.',
  'two people riding skis on a snowy surface'],
 '319154': ['A small brown dog sitting on display behind a window.',
  'A small fuzzy dog stares longingly out a window.',

### Run inference

In [12]:
import time

digit_num = 12
cnt = 0
cap_list = {}
conv_list = []

instructions_generator, _ = build_conv_instruction_prompt()

for img_id, cap in filtered_coco_caps.items():
    # Generate answers
    start_time = time.time()
    image_caption = '\n'.join(cap)
    instruct_data = instructions_generator.copy()
    instruct_data.append({"role":"user", "content":image_caption})
    prompt = format_instructions(instruct_data)
    payload = {
        "inputs": prompt,
        "parameters": {"max_new_tokens": 5000, "do_sample": True}
    }
    response = query_endpoint(payload, llm_endpoint_name)
    qa_pair = response[0]['generated_text']
    
    filtered_qa = check_qa_pair(qa_pair, image_caption, llm_endpoint_name) 
    img_id = img_id.zfill(digit_num)
    conv_list.append({
        "id": img_id,
        "image":f"coco/train2017/{img_id}.jpg",
        "conversations": parse_qa_response(filtered_qa)
    })
    print(f'{img_id} inference duration is {time.time() - start_time}')
    
    cap_list[img_id] = cap
    if cnt > 10:
        break
    cnt += 1

000000033471 inference duration is 28.87106204032898
000000052846 inference duration is 33.44689702987671
000000334872 inference duration is 42.233346939086914
000000319154 inference duration is 31.477306604385376
000000398214 inference duration is 44.37735438346863
000000520873 inference duration is 32.397223234176636
000000575173 inference duration is 37.8501341342926
000000087286 inference duration is 32.93596529960632
000000032286 inference duration is 41.73987674713135
000000175217 inference duration is 43.48673105239868
000000230238 inference duration is 46.87229871749878
000000392166 inference duration is 49.569164991378784


In [13]:
cap_list

{'000000033471': ['A white and red bus driving down a city street.',
  'A transit bus riding past some people in a parking area.',
  'A huge white bus with advertisement on the back is driving down the road.',
  'The city bus is pulled off to the side of the road. ',
  'a big bus is driving down a crowded street'],
 '000000052846': ['A cat is coming out from some curtains onto a couch.',
  'A cat is standing on the back of a couch.',
  'a couch sitting in a living room with a cat sitting on top',
  'A couch with lots of pillows and a cat sitting on top of it.',
  'A cat was on top of the couch by the red curtains'],
 '000000334872': ['A couple of people standing in the snow with skis.',
  'A pair of men with backpacks riding skis on a slope.',
  'Two men cross country skiing in the woods ',
  'Two guys on skies on a trail in the woods.',
  'two people riding skis on a snowy surface'],
 '000000319154': ['A small brown dog sitting on display behind a window.',
  'A small fuzzy dog stares

In [ ]:
parse_qa_response(test_str)

### Save result for comparison with instruct dataset generated with GPT4

In [ ]:
with open('caps_coco.json', 'w') as f:
    json.dump(cap_list, f, indent=4)

In [ ]:
import json

with open('conv_mixtral.json', 'w') as f:
    json.dump(conv_list, f, indent=4)

Go to [dataset_analyze.ipynb](dataset_analyze.ipynb) for further analysis.

## Delete endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()